In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [9]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough

llm = ChatOpenAI(
    temperature=0.1
)

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator='\n',
    chunk_size=600,
    chunk_overlap=100,
)

loader = UnstructuredFileLoader("./files/chapter_one.docx")
docs = loader.load_and_split(text_splitter=splitter)
embeddings = OpenAIEmbeddings()

# cache_dir에 있는 embed를 확인하고 업으면 OpenAIEmbeddings 사용
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    embeddings, cache_dir
)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

retriever = vectorstore.as_retriever()

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Answer questions using only the following context. If you don't know the answer just say you don't know, don't make it up:\n\n{context}"),
    ("human", "{question}")
])

chain = {"context": retriever, "question": RunnablePassthrough()} | prompt | llm

chain.invoke("Describe Victory Mansions.")

AIMessage(content='Victory Mansions is a building located in London, the chief city of Airstrip One. It is described as having glass doors at the entrance, through which Winston Smith enters. The hallway of Victory Mansions has a smell of boiled cabbage and old rag mats. There is a large colored poster tacked to the wall, depicting the face of a man in his forties with a black mustache. The building has seven floors, and Winston, who is described as a small and frail figure, slowly climbs the stairs to his flat on the seventh floor. The flat is described as having a telescreen on the right-hand wall, which cannot be completely shut off. The flat is also mentioned to have a window overlooking the street, where little eddies of wind blow dust and torn paper. Overall, Victory Mansions is portrayed as a run-down and grimy building in a decaying part of London.')